In [ ]:
import blobconverter
import cv2
import depthai as dai
import numpy as np
import time

In [ ]:
# Create pipeline
pipeline = dai.Pipeline()

# Define sources and outputs
cam_rgb = pipeline.create(dai.node.ColorCamera)
spatial_det_net = pipeline.create(dai.node.YoloSpatialDetectionNetwork)
mono_left = pipeline.create(dai.node.MonoCamera)
mono_right = pipeline.create(dai.node.MonoCamera)
stereo = pipeline.create(dai.node.StereoDepth)

xout_rgb = pipeline.create(dai.node.XLinkOut)
xout_vid = pipeline.create(dai.node.XLinkOut)
xout_nn = pipeline.create(dai.node.XLinkOut)
xout_bbdm = pipeline.create(dai.node.XLinkOut)
xout_depth = pipeline.create(dai.node.XLinkOut)

xout_rgb.setStreamName("rgb")
xout_vid.setStreamName("video")
xout_nn.setStreamName("detections")
xout_bbdm.setStreamName("boundingBoxDepthMapping")
xout_depth.setStreamName("depth")

In [ ]:
# Properties
cam_rgb.setPreviewSize(416, 416)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setVideoSize(960, 960)

cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)

mono_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# Setting node configs
stereo.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)

yolo_blob_path = "model/frozen_darknet_yolov4_model_openvino_2021.4_5shave.blob"
label_map = [
    "plates", "car"
]
spatial_det_net.setBlobPath(yolo_blob_path)
spatial_det_net.setConfidenceThreshold(0.8)
spatial_det_net.input.setBlocking(False)
spatial_det_net.setBoundingBoxScaleFactor(0.8)
spatial_det_net.setDepthLowerThreshold(100)
spatial_det_net.setDepthUpperThreshold(5000)

# Yolo specific parameters
spatial_det_net.setNumClasses(2)
spatial_det_net.setCoordinateSize(4)
spatial_det_net.setAnchors([10, 14, 23, 27, 37, 58, 81, 82, 135, 169, 344, 319])
spatial_det_net.setAnchorMasks({"side26": [1, 2, 3], "side13": [3, 4, 5]})
spatial_det_net.setIouThreshold(0.3)

In [ ]:
# Licence plate recognition
manip = pipeline.createImageManip()
manip.setWaitForConfigInput(True)

manip_img = pipeline.createXLinkIn()
manip_img.setStreamName('manip_img')
manip_img.out.link(manip.inputImage)

manip_cfg = pipeline.createXLinkIn()
manip_cfg.setStreamName('manip_cfg')
manip_cfg.out.link(manip.inputConfig)

manip_xout = pipeline.createXLinkOut()
manip_xout.setStreamName('manip_out')

tr_nn = pipeline.createNeuralNetwork()
tr_nn.setBlobPath(str(blobconverter.from_zoo(
    name='license-plate-recognition-barrier-0007',
    shaves=6,
    version='2021.4'
)))

tr_nn_xout = pipeline.createXLinkOut()
tr_nn_xout.setStreamName('recognitions')
tr_nn.out.link(tr_nn_xout.input)

In [ ]:
# Linking
mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)

cam_rgb.preview.link(spatial_det_net.input)
cam_rgb.video.link(xout_vid.input)

manip.out.link(manip_xout.input)

spatial_det_net.passthrough.link(xout_rgb.input)

spatial_det_net.out.link(xout_nn.input)
spatial_det_net.boundingBoxMapping.link(xout_bbdm.input)

stereo.depth.link(spatial_det_net.inputDepth)
spatial_det_net.passthroughDepth.link(xout_depth.input)

manip.out.link(tr_nn.input)
tr_nn.passthrough.link(manip_xout.input)

In [ ]:

# Connect to device and start pipeline
with dai.Device(pipeline) as device:
    items = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "<Anhui>", "<Beijing>", "<Chongqing>", "<Fujian>", "<Gansu>",
             "<Guangdong>", "<Guangxi>", "<Guizhou>", "<Hainan>", "<Hebei>", "<Heilongjiang>", "<Henan>", "<HongKong>",
             "<Hubei>", "<Hunan>", "<InnerMongolia>", "<Jiangsu>", "<Jiangxi>", "<Jilin>", "<Liaoning>", "<Macau>",
             "<Ningxia>", "<Qinghai>", "<Shaanxi>", "<Shandong>", "<Shanghai>", "<Shanxi>", "<Sichuan>", "<Tianjin>",
             "<Tibet>", "<Xinjiang>", "<Yunnan>", "<Zhejiang>", "<police>", "A", "B", "C", "D", "E", "F", "G", "H", "I",
             "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
    # Output queues will be used to get the rgb frames and nn data from the outputs defined above
    q_rgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    q_vid = device.getOutputQueue(name="video", maxSize=4, blocking=False)

    q_det = device.getOutputQueue(name="detections", maxSize=4, blocking=False)
    q_bbdm = device.getOutputQueue(name="boundingBoxDepthMapping", maxSize=4, blocking=False)
    q_d = device.getOutputQueue(name="depth", maxSize=4, blocking=False)

    q_rec = device.getOutputQueue(name="recognitions", maxSize=4, blocking=False)

    q_manip_img = device.getInputQueue('manip_img')
    q_manip_cfg = device.getInputQueue('manip_cfg')
    q_manip_out = device.getOutputQueue('manip_out')

    startTime = time.monotonic()
    counter = 0
    fps = 0
    color = (255, 255, 255)
    plates = np.zeros((32, 120, 3))
    rec_results = []
    while True:
        rec_stacked = None
        in_prev = q_rgb.get()
        in_vid = q_vid.get()
        in_det = q_det.get()
        depth = q_d.get()
        in_rec = q_rec.tryGet()

        if in_rec:
            rec_data = in_rec.getFirstLayerInt32()
            rec_data = rec_data[1:]
            rec_img = q_manip_out.get().getCvFrame()
            decoded_text = ""

            for idx in rec_data:
                if idx == -1:
                    break
                decoded_text += items[int(idx)]
            rec_results = [(cv2.resize(rec_img, (200, 64)), decoded_text)] + rec_results[:5]

            for rec_img, rec_text in rec_results:
                rec_placeholder_img = np.zeros((64, 200, 3), np.uint8)
                cv2.putText(rec_placeholder_img, rec_text, (5, 25), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 0))
                rec_combined = np.hstack((rec_img, rec_placeholder_img))

                if rec_stacked is None:
                    rec_stacked = rec_combined
                else:
                    rec_stacked = np.vstack((rec_stacked, rec_combined))

            if rec_stacked is not None:
                cv2.imshow("Recognized plates", rec_stacked)

        frame = in_prev.getCvFrame()
        full_frame = in_vid.getCvFrame()
        depth_frame = depth.getFrame()  # depth_frame values are in millimeters

        depth_frame_color = cv2.normalize(depth_frame, None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
        depth_frame_color = cv2.equalizeHist(depth_frame_color)
        depth_frame_color = cv2.applyColorMap(depth_frame_color, cv2.COLORMAP_HOT)

        counter += 1
        current_time = time.monotonic()

        if (current_time - startTime) > 1:
            fps = counter / (current_time - startTime)
            counter = 0
            startTime = current_time

        detections = in_det.detections

        if len(detections) != 0:
            boundingBoxMapping = q_bbdm.get()
            roiDatas = boundingBoxMapping.getConfigData()

            for roiData in roiDatas:
                roi = roiData.roi
                roi = roi.denormalize(depth_frame_color.shape[1], depth_frame_color.shape[0])
                top_left = roi.topLeft()
                bottom_right = roi.bottomRight()
                xmin = int(top_left.x)
                ymin = int(top_left.y)
                xmax = int(bottom_right.x)
                ymax = int(bottom_right.y)

                cv2.rectangle(depth_frame_color, (xmin, ymin), (xmax, ymax), color, cv2.FONT_HERSHEY_SCRIPT_SIMPLEX)

        # If the frame is available, draw bounding boxes on it and show the frame
        height = frame.shape[0]
        width = frame.shape[1]
        full_height = full_frame.shape[0]
        full_width = full_frame.shape[1]
        idx = 0
        plates_list = []

        for detection in detections:
            # Denormalize bounding box
            x1 = int(detection.xmin * width)
            x2 = int(detection.xmax * width)
            y1 = int(detection.ymin * height)
            y2 = int(detection.ymax * height)
            try:
                label = label_map[detection.label]
            except:
                label = detection.label

            if label == "plates":
                rr = dai.RotatedRect()
                x_center = int((detection.xmin + (detection.xmax - detection.xmin) / 2) * full_width)
                y_center = int((detection.ymin + (detection.ymax - detection.ymin) / 2) * full_height)
                x_size = int((detection.xmax - detection.xmin) * full_width)
                y_size = int((detection.ymax - detection.ymin) * full_height)

                rr.center.x = x_center
                rr.center.y = y_center
                rr.size.width = x_size
                rr.size.height = y_size

                cfg = dai.ImageManipConfig()
                cfg.setCropRotatedRect(rr, False)
                cfg.setResize(94, 24)
                # Send frame and config to device
                if idx == 0:
                    imgFrame = dai.ImgFrame()
                    imgFrame.setData(full_frame.transpose(2, 0, 1).flatten())
                    imgFrame.setType(dai.ImgFrame.Type.BGR888p)
                    imgFrame.setWidth(full_width)
                    imgFrame.setHeight(full_height)
                    q_manip_img.send(imgFrame)
                    idx = 1
                else:
                    cfg.setReusePreviousImage(True)
                q_manip_cfg.send(cfg)

                transformed = q_manip_out.get().getCvFrame()
                plates_list.append(transformed)
            cv2.putText(frame, str(label), (x1 + 10, y1 + 20), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 0))
            cv2.putText(frame, "{:.2f}".format(detection.confidence * 100), (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 0))
            cv2.putText(frame, f"X: {int(detection.spatialCoordinates.x)} mm", (x1 + 10, y1 + 50), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 0))
            cv2.putText(frame, f"Y: {int(detection.spatialCoordinates.y)} mm", (x1 + 10, y1 + 65), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 0))
            cv2.putText(frame, f"Z: {int(detection.spatialCoordinates.z)} mm", (x1 + 10, y1 + 80), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 255, 0))

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, cv2.FONT_HERSHEY_SIMPLEX)

        cv2.putText(frame, "NN fps: {:.2f}".format(fps), (2, frame.shape[0] - 4), cv2.FONT_HERSHEY_TRIPLEX, 0.4, color)
        if len(plates_list) > 0:
            plates_img = cv2.vconcat(plates_list)
            cv2.imshow("plates", plates_img)
        cv2.imshow("depth", depth_frame_color)
        cv2.imshow("rgb", frame)
        cv2.imshow("full_frame", full_frame)

        if cv2.waitKey(1) == ord('q'):
            break